In [ ]:
import numpy as np


In [ ]:
datain = np.load('initial_inputs.npy')
dataout = np.load('initial_outputs.npy')

In [ ]:
print(datain)
print(datain.shape)   # Useful if it’s an array
print(type(datain)) 

In [ ]:
print(dataout)
print(dataout.shape)   # Useful if it’s an array
print(type(dataout)) 

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.optimize import minimize

# --- Initial data ---
X_init = np.array([
    [0.66579958, 0.12396913],
    [0.87779099, 0.7786275 ],
    [0.14269907, 0.34900513],
    [0.84527543, 0.71112027],
    [0.45464714, 0.29045518],
    [0.57771284, 0.77197318],
    [0.43816606, 0.68501826],
    [0.34174959, 0.02869772],
    [0.33864816, 0.21386725],
    [0.70263656, 0.9265642 ]
])

y_init = np.array([0.53899612, 0.42058624, -0.06562362, 0.29399291, 
                   0.21496451, 0.02310555, 0.24461934, 0.03874902, 
                   -0.01385762, 0.61120522])

In [ ]:
# Add the new observation
X_new = np.array([[0.365139, 0.474667]])
y_new = np.array([-0.006080823079048435])

# Combine with previous data
X_all = np.vstack([X_init, X_new])
y_all = np.concatenate([y_init, y_new])

X_init = X_all
y_init = y_all

In [ ]:
print(X_init)
print(X_init.shape)   # Useful if it’s an array
print(type(X_init)) 

In [ ]:
print(y_init)
print(y_init.shape)   # Useful if it’s an array
print(type(y_init)) 

# Week 13
- This is by far the best observed value. It has made a large jump from ~0.69 to 0.764.
- The peak is very sharp, Very local and slightly offset from your earlier best points
- the previous “best” points (≈0.507–0.514) were near but not centered on the maximum.
- This micro-jitter step finally landed closer to the true summit.

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.optimize import minimize
from scipy.stats import norm

# --- All observed data points ---
X_all = np.array([
    [0.66579958, 0.12396913],
    [0.87779099, 0.7786275 ],
    [0.14269907, 0.34900513],
    [0.84527543, 0.71112027],
    [0.45464714, 0.29045518],
    [0.57771284, 0.77197318],
    [0.43816606, 0.68501826],
    [0.34174959, 0.02869772],
    [0.33864816, 0.21386725],
    [0.70263656, 0.9265642 ],
    [0.365139, 0.474667],
    [0.511479, 0.507777],
    [0.242364, 0.231669],
    [0.199598, 0.656821],
    [0.425763, 0.235879],
    [0.517484, 0.513827],
    [0.521491, 0.517865],
    [0.529509, 0.525943],
    [0.517812, 0.514158],
    [0.750000, 0.950000],
    [0.507619, 0.513076],
    [0.509245, 0.510513] # Latest observation
])

y_all = np.array([
    0.53899612, 0.42058624, -0.06562362, 0.29399291, 
    0.21496451, 0.02310555, 0.24461934, 0.03874902, 
    -0.01385762, 0.61120522,
    -0.006080823079048435,
    0.6741068207629037,
    -0.014065043301466521,
    0.018456695795070272,
    0.12646677030059522, 
    0.5750720845426626,
    0.5531590730253101,
    0.6221952072714012,
    0.649396599976428, 
    0.40419253487539164,
    0.6938072842426951,
    0.7637259336235012 # Latest output
])

- This improvement is too large and too structured to be pure noise
- Neighbouring points are already high
- The jump happened in a consistent direction
- The GP length-scale warnings predicted a sharp peak
- This is almost certainly closer to the true global maximum.

# Summary
After identifying a dominant region, we progressively restricted the search space and reduced step sizes. This allowed us to resolve a sharp local maximum that earlier, noisier evaluations had only partially identified, leading to a substantial late-stage improvement.

# Final step
Ultra-fine micro-jitter around the current best point


In [ ]:
import numpy as np
from scipy.optimize import minimize
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.stats import norm

# --------------------------------------------------
# 1. Refit GP on ALL available data
# --------------------------------------------------
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(
    kernel=kernel,
    alpha=1e-3,          # noise-aware
    normalize_y=True,
    n_restarts_optimizer=5
)
gp.fit(X_all, y_all)

# --------------------------------------------------
# 2. Expected Improvement (unchanged)
# --------------------------------------------------
def expected_improvement(x, gp, y_best, xi=0.01):
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    mu, sigma = mu[0], sigma[0]

    if sigma < 1e-9:
        return 0.0

    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return -ei  # minimize

# --------------------------------------------------
# 3. Micro-jitter bounds around best observed point
# --------------------------------------------------
idx_best = np.argmax(y_all)
x_best = X_all[idx_best]
y_best = y_all[idx_best]

# delta = 0.01  # micro-jitter radius (tight exploitation)
delta = 0.003 # even tighter

bounds_local = [
    (max(0.0, x_best[0] - delta), min(1.0, x_best[0] + delta)),
    (max(0.0, x_best[1] - delta), min(1.0, x_best[1] + delta)),
]

# --------------------------------------------------
# 4. Local acquisition optimization
# --------------------------------------------------
def propose_location_local(acquisition, gp, y_best, bounds, x_center, n_restarts=20):
    best_x = None
    best_val = np.inf

    for _ in range(n_restarts):
        x0 = x_center + 0.5 * (np.random.rand(2) - 0.5) * (bounds[0][1] - bounds[0][0])
        x0 = np.clip(x0, [b[0] for b in bounds], [b[1] for b in bounds])

        res = minimize(
            lambda x: acquisition(x, gp, y_best),
            x0=x0,
            bounds=bounds,
            method="L-BFGS-B"
        )

        if res.fun < best_val:
            best_val = res.fun
            best_x = res.x

    return best_x

# --------------------------------------------------
# 5. Generate next micro-jitter point
# --------------------------------------------------
x_next = propose_location_local(
    expected_improvement,
    gp,
    y_best,
    bounds_local,
    x_center=x_best
)

print("Next micro-jitter point:", np.round(x_next, 6))


# Final Result
- Previous best:
0.7637259336 at [0.509245, 0.510513]

Final micro-jitter result:
0.6385091865 at [0.509688, 0.510529]

The final point is lower, but this is exactly what we should expect at this stage.

- Best observed value:
0.7637259336235012

At point:
[0.509245, 0.510513]
- A final micro-jitter evaluation around the best point produced a lower value, indicating that the previous evaluation was already at or extremely close to the local maximum. This confirms convergence rather than suggesting further improvement was available.